In [1]:

import sys

sys.path.append("../src")


import pandas as pd
import numpy as np
import pathlib
import random
import itertools
from sklearn import metrics
from tqdm.auto import tqdm

import math
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MultiLabelBinarizer

from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from sklearn.feature_extraction.text import CountVectorizer

from sentence_transformers import SentenceTransformer

import os
from transformers import DistilBertPreTrainedModel, PretrainedConfig, DistilBertModel
from transformers import DistilBertConfig
from transformers.models.distilbert.modeling_distilbert import SequenceClassifierOutput
from transformers import DistilBertTokenizer
import torch
from torch import nn
from typing import Optional
from torch.nn import MSELoss, CrossEntropyLoss
from typing import Union
from typing import Tuple
from scipy.special import softmax

# train model
from transformers import TrainingArguments, Trainer
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import average_precision_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import auc
from sklearn.metrics import roc_curve

from utils import number_split, create_mix
from data_process import load_wls_adress_AddDomain
from process_SHAC import load_process_SHAC
from custom_distance import KL

import pickle

from transformers import AutoTokenizer, AutoModel
import torch.nn.functional as F



In [84]:

# (1) dynGen
df_dynGen = pd.read_csv(
    "/bime-munin/xiruod/data/hateSpeech_Bulla2023/Dynamically-Generated-Hate-Speech-Dataset/Dynamically Generated Hate Dataset v0.2.3.csv",
)

df_dynGen["label"] = df_dynGen["label"].map({"hate": "hate", "nothate": "nothate"})
df_dynGen["dfSource"] = "dynGen"
df_dynGen["label_binary"] = df_dynGen["label"].map({"hate": 1, "nothate": 0})

# (2)  wsf
ls_allFiles = pathlib.Path(
    "/bime-munin/xiruod/data/hateSpeech_Bulla2023/hate-speech-dataset/all_files/"
).glob("*.txt")

ls_id = []
ls_text = []

for ifile in ls_allFiles:
    ls_id.append(ifile.name.split(".txt")[0])
    with open(ifile, "r") as f:
        ls_text.append(f.read())

df_wsf_raw = pd.DataFrame({"file_id": ls_id, "text": ls_text})

df_wsf_annotation = pd.read_csv(
    "/bime-munin/xiruod/data/hateSpeech_Bulla2023/hate-speech-dataset/annotations_metadata.csv"
)

df_wsf = df_wsf_raw.merge(df_wsf_annotation, on="file_id", how="inner")

df_wsf = df_wsf[df_wsf["label"].isin(["hate", "noHate"])].reset_index(drop=True)

# df_wsf['label_binary'] = df_wsf['label'].map({"hate":1,"noHate":0})
df_wsf["label"] = df_wsf["label"].map({"hate": "hate", "noHate": "nothate"})
df_wsf["dfSource"] = "wsf"
df_wsf["label_binary"] = df_wsf["label"].map({"hate": 1, "nothate": 0})


In [85]:
### Hate Speech
z_Categories = ["dynGen", "wsf"]  # the order here matters! Should match with df0, df1
label = "label_binary"
n_zCats = len(z_Categories)
txt_col = "text"
domain_col = "dfSource"
df0 = df_dynGen
df1 = df_wsf

In [86]:


from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

df0["labels"] = le.fit_transform(df0[label])
df1["labels"] = le.fit_transform(df1[label])

tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
tokenizer.max_len = 512

In [221]:


# add labels and source to data for BERT
for _ in [df0, df1]:
    _["tokenized"] = _[txt_col].apply(
        lambda x: tokenizer(
            x,
            return_tensors="pt",
            truncation=True,
            padding="max_length",
            max_length=512,
        )
    )
    for i, row in _.iterrows():
        row["tokenized"].update(
            {"labels": torch.tensor(int(row["labels"]), dtype=torch.long)}
        )
        row["tokenized"].update(
            {
                "source": torch.tensor(
                    int(row[domain_col] == z_Categories[0]), dtype=torch.long
                )
            }
        )  # NOTE: this only works for binary Category cases!!!
        row["tokenized"].update({"input_ids": row["tokenized"]["input_ids"].squeeze()})
        row["tokenized"].update(
            {"attention_mask": row["tokenized"]["attention_mask"].squeeze()}
        )


In [222]:

##### Split
# Hate Speech Detection: df_dynGen (0.55) vs df_wsf (0.11)
n_test = 1000
train_test_ratio = 4


p_pos_train_z0_ls = [
    0.2,
    0.4,
    0.5,
    0.6,
    0.8,
    0.9,
]  # probability of training set examples drawn from site/domain z0 being positive
p_pos_train_z1_ls = [
    0.2,
    0.4,
    0.5,
    0.6,
    0.8,
    0.9,
]  # probability of test set examples drawn from site/domain z1 being positive

p_mix_z1_ls = [0.2, 0.4, 0.6, 0.8]  # = np.arange(0.1, 0.9, 0.05)

# alpha_test_ls = np.arange(0, 10, 0.05)

numvals = 1023
base = 1.1
alpha_test_ls = np.power(base, np.arange(numvals)) / np.power(base, numvals // 2)


valid_full_settings = []
for combination in itertools.product(
    p_pos_train_z0_ls, p_pos_train_z1_ls, p_mix_z1_ls, alpha_test_ls
):
    number_setting = number_split(
        p_pos_train_z0=combination[0],
        p_pos_train_z1=combination[1],
        p_mix_z1=combination[2],
        alpha_test=combination[3],
        train_test_ratio=train_test_ratio,
        n_test=n_test,
        verbose=False,
    )

    if number_setting is not None:
        # if not (round(number_setting['mix_param_dict']['C_y'], 4) in [0.36, 0.44, 0.52]):
        #     continue
        if np.all([number_setting[k] >= 10 for k in list(number_setting.keys())[:-1]]):
            valid_full_settings.append(number_setting)



In [223]:
valid_n_full_settings = []

for c in tqdm(valid_full_settings):
    # for c in test_settings:

    c = c.copy()

    # create train/test split according to stats
    # dfs = create_mix(df0=df_wls_merge, df1=df_adress, target='label', setting= c, sample=False)
    # dfs = create_mix(df0=df_shac_uw, df1=df_shac_mimic, target=label, setting= c, sample=False, seed=random.randint(0,1000))
    dfs = create_mix(
        df0=df0,
        df1=df1,
        target=label,
        setting=c,
        sample=False,
        # seed=random.randint(0,1000),
        seed=222,
    )

    if dfs is None:
        continue

    # #### TO DELETE: For results on Selected C_y ONLY!!!!!!!!!
    # if round(c['mix_param_dict']['C_y'], 4) not in [0.36, 0.44, 0.52, 0.24, 0.54, 0.84]:
    #     continue

    c["run"] = 3
    valid_n_full_settings.append(c)

    y_train = dfs["train"][label]
    y_test = dfs["test"][label]

    n_test = len(y_test)
    
    df_test = dfs["test"].copy(deep=True)
 
    break

  0%|          | 0/5642 [00:00<?, ?it/s]

In [224]:
tmp = set()
for i, row in df_test.iterrows():
    tmp.add(row['tokenized']['source'].tolist())

In [225]:
tmp

{0, 1}

In [226]:
df_test.iloc[0]

Unnamed: 0                                                  11555.0
acl.id                                                     acl20397
X1                                                          11555.0
text              Last nght I saw some japanese people walking o...
label                                                          hate
type                                                      animosity
target                                                     asi.east
level                                                      original
split                                                           dev
round.base                                                      2.0
annotator                                                eLGzdD8Tvb
round                                                            2a
acl.id.matched                                             acl19953
dfSource                                                     dynGen
label_binary                                    

In [227]:
df_test.iloc[12]['tokenized']

{'input_ids': tensor([  101,  1045,  2052,  6235,  2032,  2004,  1037,  9346, 15667,  3054,
        18150,  2007, 11238, 28583,  2015,  2725, 10124,  2923,  2731,   102,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0, 

In [229]:
from copy import deepcopy

In [231]:
for i in range(5):
    df_test[f'tokenized_sr{i}'] = df_test.apply(lambda x: deepcopy(x["tokenized"]), axis=1)
    for _, row in df_test.iterrows():
        row[f'tokenized_sr{i}']['source'] = torch.tensor(i, dtype=torch.long)


In [232]:
id(df_test.iloc[1]['tokenized_sr0'])

140477605855184

In [233]:
id(df_test.iloc[1]['tokenized_sr1'])

140477732757616

In [234]:
id(df_test.iloc[1]['tokenized'])

140478762427056

In [235]:
df_test.iloc[0]['tokenized_sr1']['source']

tensor(1)

In [236]:
df_test.iloc[0]['tokenized_sr0']['source']

tensor(0)

In [237]:
df_test.iloc[0]['tokenized']['source']

tensor(1)

In [237]:
df_test.iloc[0]['tokenized']['source']

tensor(1)

In [241]:
tmp = set()
for i, row in df_test.iterrows():
    tmp.add(row['tokenized_sr4']['source'].tolist())
tmp

{4}